# Imports and URLs

In [2]:
import pandas as pd

titlebasicsURL = "https://datasets.imdbws.com/title.basics.tsv.gz"
titleakasURL = "https://datasets.imdbws.com/title.akas.tsv.gz"
titleratingsURL = "https://datasets.imdbws.com/title.ratings.tsv.gz"

In [6]:
basics = pd.read_csv(titlebasicsURL, sep='\t', low_memory=False)
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
9708082,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010,\N,\N,"Action,Drama,Family"
9708083,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,"Action,Drama,Family"
9708084,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family"
9708085,tt9916856,short,The Wind,The Wind,0,2015,\N,27,Short


In [9]:
akas = pd.read_csv(titleakasURL, sep='\t', low_memory=False)
akas

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0
...,...,...,...,...,...,...,...,...
35320158,tt9916852,5,Episódio #3.20,PT,pt,\N,\N,0
35320159,tt9916852,6,Episodio #3.20,IT,it,\N,\N,0
35320160,tt9916852,7,एपिसोड #3.20,IN,hi,\N,\N,0
35320161,tt9916856,1,The Wind,DE,\N,imdbDisplay,\N,0


In [10]:
ratings = pd.read_csv(titleratingsURL, sep='\t', low_memory=False)
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1960
1,tt0000002,5.8,263
2,tt0000003,6.5,1799
3,tt0000004,5.6,179
4,tt0000005,6.2,2598
...,...,...,...
1292294,tt9916730,8.3,10
1292295,tt9916766,7.0,21
1292296,tt9916778,7.2,36
1292297,tt9916840,8.8,6
